In [ ]:
#import os # only lib needed to cut the files
#import glob # these two for splitting
#import shutil
#%config Completer.use_jedi = False # for auto complete to work this option needs to be disabled.

## `Pre-Processing images`

***

### Move all images out of subfolders into 1 folder

In [ ]:
# get images of the scans with cancer cells and cuts it to a folder, only need this to run ONCE.

#src_dir = r'D:\Breast Cancer Diagnosis\Dataset\Cancer' + '\\' # source folder with subfolders that contain the images
#target_dir = r'D:\trimmed_dataset\Cancer'+ '\\' # destination which i am copying the files to

#for path, dir, files in os.walk(src_dir): # loop through all files in the source folder
    #if files: # subfolders
        #for img in files: # loop through every image in every subfolder
            #if not os.path.isfile(target_dir + img): #if not a duplicate
                #os.rename(path + '\\' + img, target_dir + img) # cut to new folder
        

In [ ]:
# get images of the scans with normal cells and cuts it to a folder, only need this to run ONCE.

#src_dir = r'D:\Breast Cancer Diagnosis\Dataset\Normal' + '\\' # source folder with subfolders that contain the images
#target_dir = r'D:\trimmed_dataset\Normal'+ '\\' # destination which i am copying the files to

#for path, dir, files in os.walk(src_dir): # loop through all files in the source folder
    #if files: # subfolders
        #for img in files: # loop through every image in every subfolder
            #if not os.path.isfile(target_dir + img): #if not a duplicate
                #os.rename(path + '\\' + img, target_dir + img) # cut to new folder

### Split into Test and Train Folders

In [ ]:
# split the Cancer folder into train folder and test folders.

# Train Folder is 80% of total imgs i.e. 2,172 images.

#src_dir = r'D:\trimmed_dataset\Cancer' + '//'
#target_dir = r'D:\trimmed_dataset\Train\Cancer' + '//'

#count = 0

#for img in glob.iglob(os.path.join(src_dir, "*.png")):
    #shutil.move(img, target_dir)
    #count += 1
    #if (count == 2172):
        #break

# Test Folder is 20% of total images i.e 544 images.

#src_dir = r'D:\trimmed_dataset\Cancer' + '//'
#target_dir = r'D:\trimmed_dataset\Test\Cancer' + '//'

#count = 0

#for img in glob.iglob(os.path.join(src_dir, "*.png")):
    #shutil.move(img, target_dir)
    #count += 1
    #if (count == 544):
        #break


In [ ]:
# split the Cancer folder into train folder and test folders.

# Train Folder is 80% of total imgs i.e. 1,926 images.

#src_dir = r'D:\trimmed_dataset\Normal' + '//'
#target_dir = r'D:\trimmed_dataset\Train\Normal' + '//'

#count = 0

#for img in glob.iglob(os.path.join(src_dir, "*.png")):
    #shutil.move(img, target_dir)
    #count += 1
    #if count == 1926:
        #break
        
        
# Test Folder is 20% of total images i.e 482 images.

#src_dir = r'D:\trimmed_dataset\Normal' + '//'
#target_dir = r'D:\trimmed_dataset\Test\Normal' + '//'

#count = 0

#for img in glob.iglob(os.path.join(src_dir, "*.png")):
    #shutil.move(img, target_dir)
    #count += 1
    #if (count == 482):
        #break


## `Breast Cancer Detection`

***

In [ ]:
%config Completer.use_jedi = False 
import os
os.listdir('D:/trimmed_dataset/')

### Import libs.

In [2]:
from keras.layers import Input, Lambda, Dense, Flatten 
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import os

### Resize images.

In [4]:
IMG_SIZE = [224, 224]

train_path = 'D:/trimmed_dataset/Train'
test_path = 'D:/trimmed_dataset/Test'

### Model creation with VGG16

In [5]:
model = VGG16(input_shape = IMG_SIZE + [3], weights = 'imagenet', include_top = False)

A local file was found, but it seems to be incomplete or outdated because the auto file hash does not match the original value of 6d6bbae143d832006294945121d1f1fc so we will re-download the data.
58892288/58889256 [==============================] - 485s 8us/step


### Skip the trained weights

In [6]:
for layer in model.layers:
    layer.trainable = False

### Get output classes

In [8]:
folders = glob(train_path + '/*')
folders

['D:/trimmed_dataset/Train\\Cancer', 'D:/trimmed_dataset/Train\\Normal']

### Make flatten layer to flatten current VGG output.

In [9]:
x = Flatten()(model.output)

### Adding last layer

In [10]:
prediction = Dense(len(folders), activation='softmax')(x)

### Create a model object

In [11]:
model_obj = Model(inputs=model.input, outputs=prediction)

### The summary of the model.

In [12]:
model_obj.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

### Specifying cost and optimization methods.

In [13]:
model_obj.compile(
    loss='categorical_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy']
)

### Use Image Data Generator to import the images.

In [14]:
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

In [15]:
training_set = train_datagen.flow_from_directory(train_path, target_size=(224,224), batch_size=32, class_mode='categorical')

Found 4098 images belonging to 2 classes.


In [16]:
training_set.class_indices

{'Cancer': 0, 'Normal': 1}

In [17]:
testing_set = test_datagen.flow_from_directory(test_path, target_size=(224,224), batch_size=32, class_mode='categorical')

Found 1026 images belonging to 2 classes.


In [18]:
testing_set.class_indices

{'Cancer': 0, 'Normal': 1}